<a href="https://colab.research.google.com/github/Ravikrishnan05/PrediscanMedtech_project/blob/main/Final_with_Documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom InceptionV3 Model with Attention Mechanism

In this code, we are building a custom model using **InceptionV3** as the backbone, with additional custom layers for specific tasks, such as applying attention mechanisms and performing classification.

### 1. **Import Libraries**
The required libraries are imported:
- **Keras**: For building the neural network model and loading the pre-trained InceptionV3.
- **TensorFlow**: To provide the necessary computational backend for training and inference.
- **NumPy**: Used for handling arrays and initializing certain layers with custom weights.

In [5]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, Conv2D, multiply, Lambda, BatchNormalization
from keras.models import Model
import tensorflow as tf
from keras.utils import get_source_inputs

### 2. **Defining Input Shape**
Here, we specify the input shape for the model. In this case, the input shape is `(512, 512, 3)`, which means the model expects images of size `512x512` with 3 color channels (RGB). You can adjust the dimensions based on the dataset.


In [6]:
# Set input shape (e.g., from your data)
input_shape = (512, 512, 3)

### 3. **Loading Pre-trained InceptionV3 Model**
We load the **InceptionV3** model, excluding its top layers (the fully connected layers), and initialize the weights from the **ImageNet** dataset. This provides the model with a good starting point for learning from image data. The model is configured to exclude the top layers so we can add our custom layers on top for specific tasks like attention mechanisms and classification.


In [7]:
# Create a new InceptionV3 model with the same configuration as the original
base_pretrained_model = InceptionV3(
    input_shape=input_shape, include_top=False, weights='imagenet'
)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


### 4. **Freezing the Base Model Layers**
We freeze the layers of the pre-trained **InceptionV3** model. This ensures that the learned weights from ImageNet remain unchanged during training. Freezing helps speed up the training process and prevents overfitting by reusing features learned from a large dataset like ImageNet.


In [8]:
in_lay = get_source_inputs(base_pretrained_model.output)[0]

base_pretrained_model.trainable = False
pt_depth = base_pretrained_model.output_shape[-1]

pt_features = base_pretrained_model(in_lay)

### 5. **Adding Custom Layers**
After loading the pre-trained model, we add several custom layers on top:
- **BatchNormalization**: To normalize the feature maps coming from the InceptionV3 model. This helps with training stability and faster convergence.
- **Dropout**: Used to prevent overfitting by randomly setting a fraction of input units to 0 during training.
- **Conv2D**: 1x1 convolutions to further process the feature maps from InceptionV3. These layers help the model to learn more specialized features.
- **Multiply**: We use this layer to implement an attention mechanism by multiplying the features with a learned attention map, allowing the model to focus on important regions.


In [9]:
# Apply your custom layers on top of InceptionV3
bn_features = BatchNormalization()(pt_features)
x = Dropout(0.5, name='dropout_1')(bn_features)
x = Conv2D(64, kernel_size=(1, 1), padding='same', activation='relu', name='conv2d_95')(x)
x = Conv2D(16, kernel_size=(1, 1), padding='same', activation='relu', name='conv2d_96')(x)
x = Conv2D(8, kernel_size=(1, 1), padding='same', activation='relu', name='conv2d_97')(x)
x = Conv2D(1, kernel_size=(1, 1), padding='valid', activation='sigmoid', name='conv2d_98')(x)

### 6. **Applying Attention Mechanism**
We create a custom attention mechanism using the **Conv2D** layer. The weight of the attention map is initialized to ones, and then it is applied to the feature map from the InceptionV3 model to selectively focus on important regions.


In [10]:
up_c2_w = np.ones((1, 1, 1, pt_depth), dtype=np.float32)
up_c2 = Conv2D(pt_depth, kernel_size=(1, 1), padding='same', activation='linear', use_bias=False, name='conv2d_99')
up_c2.build((None, *x.shape[1:]))
up_c2.set_weights([up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(x)

### 7. **Global Average Pooling**
The feature maps are passed through a **GlobalAveragePooling2D** layer, which reduces the spatial dimensions while retaining essential information. This step summarizes the feature maps into a single vector per image.


In [11]:
mask_features = multiply([attn_layer, bn_features], name='multiply_1')
gap_features = GlobalAveragePooling2D(name='global_average_pooling2d_1')(mask_features)
gap_mask = GlobalAveragePooling2D(name='global_average_pooling2d_2')(attn_layer)
gap = Lambda(lambda x: x[0] / x[1], name='RescaleGAP')([gap_features, gap_mask])


### 8. **Dense Layers and Output Layer**
After the pooling step, we use a dense layer with **Dropout** for regularization and then add the final output layer with 5 units and a **softmax** activation function. This gives the model a probability distribution over 5 classes for classification tasks.


In [12]:
gap_dr = Dropout(0.25, name='dropout_2')(gap)
dr_steps = Dropout(0.25, name='dropout_3')(Dense(128, activation='relu', name='dense_1')(gap_dr))
out_layer = Dense(5, activation='softmax', name='dense_2')(dr_steps)

### 9. **Output layer**
We then define the Model by specifying the input and output layers. This model is the final architecture, combining the InceptionV3 backbone with the custom layers.


In [13]:
retina_model = Model(inputs=[in_lay], outputs=[out_layer])

### 10. **Loading Weights**
The model weights are loaded from a pre-trained `.h5` file. The `by_name=True` argument ensures that the weights are loaded into the corresponding layers by matching their names. We also use `skip_mismatch=True` to ignore any mismatches between the saved weights and the current model architecture.

In [14]:
retina_model.load_weights('/content/full_retina_model (2).h5', by_name=True, skip_mismatch=True)

/usr/local/lib/python3.11/dist-packages/keras/src/legacy/saving/legacy_h5_format.py:513: UserWarning: Skipping loading weights for layer #1 (named inception_v3)due to mismatch in shape for weight batch_normalization/moving_mean. Weight expects shape (32,). Received saved weight with shape (3, 3, 32, 32)
  _set_weights(
/usr/local/lib/python3.11/dist-packages/keras/src/legacy/saving/legacy_h5_format.py:513: UserWarning: Skipping loading weights for layer #1 (named inception_v3)due to mismatch in shape for weight conv2d_1/kernel. Weight expects shape (3, 3, 32, 32). Received saved weight with shape (3, 3, 32, 64)
  _set_weights(
/usr/local/lib/python3.11/dist-packages/keras/src/legacy/saving/legacy_h5_format.py:513: UserWarning: Skipping loading weights for layer #1 (named inception_v3)due to mismatch in shape for weight batch_normalization_1/beta. Weight expects shape (32,). Received saved weight with shape (64,)
  _set_weights(
/usr/local/lib/python3.11/dist-packages/keras/src/legacy/s

### 11. **Re-initializing Specific Weights**
In this section, we re-initialize the weights for some layers. This is useful when you want to reset specific layers (e.g., InceptionV3 layers) while keeping others intact. Here, the weights of layers that are not BatchNormalization layers are reset to zero.

In [15]:
# Then, re-initialize the weights of the InceptionV3 layers
for layer in base_pretrained_model.layers:
    if layer.name in retina_model.layers and layer.get_weights(): # Check if the layer has weights
        if isinstance(layer, BatchNormalization): # Only re-initialize non-batch normalization layers
            continue
        weights = layer.get_weights()
        # You can use different initialization strategies (e.g., random_normal, glorot_uniform)
        # but here we'll just reset to zeros for demonstration
        weights = [np.zeros(w.shape) for w in weights]
        layer.set_weights(weights)
        print(f"Re-initialized weights for layer: {layer.name}")

In [16]:
retina_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_v3        │ (None, 14, 14,    │ 21,802,784 │ input_layer[0][0] │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │      8,192 │ inception_v3[0][… │
│ (BatchNormalizatio… │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 14, 14,    │          0 │ batch_normalizat… │
│                     │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 14, 14,    │    131,136 │ dropout_1[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 14, 14,    │      1,040 │ conv2d_95[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 14, 14, 8) │        136 │ conv2d_96[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 14, 14, 1) │          9 │ conv2d_97[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_99 (Conv2D)  │ (None, 14, 14,    │      2,048 │ conv2d_98[0][0]   │
│                     │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 14, 14,    │          0 │ conv2d_99[0][0],  │
│ (Multiply)          │ 2048)             │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ multiply_1[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ conv2d_99[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RescaleGAP (Lambda) │ (None, 2048)      │          0 │ global_average_p… │
│                     │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 2048)      │          0 │ RescaleGAP[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │    262,272 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 5)         │        645 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 22,208,262 (84.72 MB)

 Trainable params: 399,334 (1.52 MB)

 Non-trainable params: 21,808,928 (83.19 MB)

In [17]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [18]:
for layer in retina_model.layers:
    weights = layer.get_weights()
    if weights:
        total_weight_sum = sum([np.sum(np.abs(w)) for w in weights])
        print(f"{layer.name}: Weights loaded, sum = {total_weight_sum:.2f}")
    else:
        print(f"{layer.name}: No weights.")

input_layer: No weights.
inception_v3: Weights loaded, sum = 365519.88
batch_normalization_94: Weights loaded, sum = 4096.00
dropout_1: No weights.
conv2d_95: Weights loaded, sum = 3536.01
conv2d_96: Weights loaded, sum = 139.78
conv2d_97: Weights loaded, sum = 32.41
conv2d_98: Weights loaded, sum = 3.60
conv2d_99: Weights loaded, sum = 2048.00
multiply_1: No weights.
global_average_pooling2d_1: No weights.
global_average_pooling2d_2: No weights.
RescaleGAP: No weights.
dropout_2: No weights.
dense_1: Weights loaded, sum = 7012.47
dropout_3: No weights.
dense_2: Weights loaded, sum = 69.10


In [19]:
import zipfile
import os

zip_path = '/content/sample.zip'
extract_path = '/content/sample_images/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [20]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

# Helper function to load and preprocess a single image for InceptionV3
def preprocess_image(img_path, target_size=(512, 512)):
    img = image.load_img(img_path, target_size=target_size)  # Resize to input_shape
    img_array = image.img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension: (1, 512, 512, 3)
    img_array = tf.keras.applications.inception_v3.preprocess_input(img_array)  # Apply InceptionV3 preprocessing
    return img_array

In [21]:
import glob

image_paths = glob.glob(os.path.join(extract_path, '**', '*.*'), recursive=True)
image_paths = [p for p in image_paths if p.lower().endswith(('.png', '.jpg', '.jpeg'))]



In [23]:
# Step 4: Load and preprocess images
images = np.concatenate([preprocess_image(p) for p in image_paths], axis=0) # Change here

# Step 5: Predict using your loaded model
predictions = retina_model.predict(images)

# Step 6: Show predictions
predicted_classes = np.argmax(predictions, axis=1)

for path, pred_class in zip(image_paths, predicted_classes):
    print(f"{os.path.basename(path)} → Predicted Class: {pred_class}")

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(10, 512, 512, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
17_right.jpeg → Predicted Class: 0
17_left.jpeg → Predicted Class: 0
10_right.jpeg → Predicted Class: 0
16_left.jpeg → Predicted Class: 0
15_left.jpeg → Predicted Class: 0
10_left.jpeg → Predicted Class: 0
15_right.jpeg → Predicted Class: 0
13_right.jpeg → Predicted Class: 0
16_right.jpeg → Predicted Class: 0
13_left.jpeg → Predicted Class: 0


In [24]:
class_names = ["Normal", "Mild", "Moderate", "Severe", "Proliferative"]

for path, pred_class in zip(image_paths, predicted_classes):
    print(f"{os.path.basename(path)} → Predicted Class: {class_names[pred_class]}")

17_right.jpeg → Predicted Class: Normal
17_left.jpeg → Predicted Class: Normal
10_right.jpeg → Predicted Class: Normal
16_left.jpeg → Predicted Class: Normal
15_left.jpeg → Predicted Class: Normal
10_left.jpeg → Predicted Class: Normal
15_right.jpeg → Predicted Class: Normal
13_right.jpeg → Predicted Class: Normal
16_right.jpeg → Predicted Class: Normal
13_left.jpeg → Predicted Class: Normal
